# Databases - week11
install packages: mysql-connector-python

In [ ]:
! pip install mysql-connector-python

In [3]:
#regular syntax of connecting to a database
import mysql.connector

dbconnection = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = "",
    database = "movies_db"
)

cursor = dbconnection.cursor()
print(dbconnection.database)
#do stuff
cursor.close()
dbconnection.close()

movies_db


## CREATE OPERATION

In [4]:
#CRUD CREATE READ UPDATE DELETE
#creating extra data --> create operation
import mysql.connector

dbconnection = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = "",
    database = "movies_db"
)

cursor = dbconnection.cursor()
query = "INSERT INTO actors(name) VALUES('Kevin Hart');"
cursor.execute(query)
dbconnection.commit()
cursor.close()
dbconnection.close()


In [5]:
import mysql.connector

dbconnection = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = "",
    database = "movies_db"
)

cursor = dbconnection.cursor()
query = "INSERT INTO actors(name) VALUES(%s);"
data = [["Leondardo Dicaprio"], ["Samuel L Jackson"], ["Angelina Jolie"], 
        ["Benedict Cumberbatch"], ["Margot Robbie"]]
cursor.executemany(query, data)

#also possible if a category in db has multiple values
query = "INSERT INTO movies(title, synopsis) VALUES(%s, %s);"
data = ("Shreck", "An ogre that falls in love with a princess, travels through the country with a talking donkey")

cursor.execute(query, data)

movies = [("Jurassic Park", "Scientists create a theme park with cloned dinosaurs, which goes wrong"),
          ("Ratatouille", "A rat with cooking skills helping a crapping chef become the best in Paris"),
          ("Titanic", "Ship crashing into iceberg, everyone dies but Rose")]

#insert each movie
for movie in movies:
    cursor.execute(query, movie)

dbconnection.commit()
cursor.close()
dbconnection.close()


## READ OPERATION

In [6]:
import mysql.connector

dbconnection = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = "",
    database = "movies_db"
)

cursor = dbconnection.cursor()
query = "SELECT * FROM actors;"
cursor.execute(query) #cursor.execute("SELECT * FROM actors;")
data = cursor.fetchall()
print(data) #all of it no format

for actor in data:
    print(actor) #print entire record

for actor in data:
    print(actor[1]) # 0 = ID, 1 = name

for id, name in data:
    print(id) #print(id, name)

cursor.close()
dbconnection.close()

[(1, 'Michelle Yeoh'), (2, 'Stephan James'), (3, 'Jamie Lee Curtis'), (4, 'Tom Cruise'), (5, 'Kevin Hart'), (6, 'Leondardo Dicaprio'), (7, 'Samuel L Jackson'), (8, 'Angelina Jolie'), (9, 'Benedict Cumberbatch'), (10, 'Margot Robbie')]
(1, 'Michelle Yeoh')
(2, 'Stephan James')
(3, 'Jamie Lee Curtis')
(4, 'Tom Cruise')
(5, 'Kevin Hart')
(6, 'Leondardo Dicaprio')
(7, 'Samuel L Jackson')
(8, 'Angelina Jolie')
(9, 'Benedict Cumberbatch')
(10, 'Margot Robbie')
Michelle Yeoh
Stephan James
Jamie Lee Curtis
Tom Cruise
Kevin Hart
Leondardo Dicaprio
Samuel L Jackson
Angelina Jolie
Benedict Cumberbatch
Margot Robbie
1
2
3
4
5
6
7
8
9
10


In [11]:
import mysql.connector

dbconnection = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = "",
    database = "movies_db"
)

cursor = dbconnection.cursor()
query = """select movies.title, actors.name from movies
                inner join movie_actor on movies.id = movie_actor.movie_id
                inner join actors on actors.id = movie_actor.actor_id;"""
#can also do left outer join --> not all movies have actor linked
cursor.execute(query)
data = cursor.fetchall()

for row in data:
    print(row)

for movie, actor in data:
    print(actor, "plays in", movie)

cursor.close()
dbconnection.close()

('Everything Everywhere All at Once', 'Michelle Yeoh')
('Everything Everywhere All at Once', 'Stephan James')
('Everything Everywhere All at Once', 'Jamie Lee Curtis')
('Top Gun: Maverick', 'Tom Cruise')
Michelle Yeoh plays in Everything Everywhere All at Once
Stephan James plays in Everything Everywhere All at Once
Jamie Lee Curtis plays in Everything Everywhere All at Once
Tom Cruise plays in Top Gun: Maverick


## UPDATE OPERATION

In [13]:
#let's add home alone without descr. --> update later with descr
import mysql.connector
dbconnection = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = "",
    database = "movies_db"
)

cursor = dbconnection.cursor()

#cursor.execute("INSERT INTO movies(title) VALUES('Home Alone')")
query = """UPDATE movies SET synopsis = 
            'A kid has to defend his home from 2 burglars after he got left home alone unknowingly'
            WHERE title like 'Home Alone'"""
cursor.execute(query)

dbconnection.commit()
cursor.close()
dbconnection.close()

In [18]:
import requests
from bs4 import BeautifulSoup

url = "https://en.wikipedia.org/wiki/List_of_Academy_Award-winning_films"
response = requests.get(url)
soup = BeautifulSoup(response.text)

#import db
import mysql.connector
dbconnection = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = "",
    database = "movies_db"
)

cursor = dbconnection.cursor()
table = soup.find("table")
rows = table.find_all("tr")

for row in rows:
    td = row.find_all("td")
    if len(td) > 0:
        title = td[0].get_text().strip().replace("'", "")
        year = td[1].get_text().strip()
        awards = td[2].get_text().strip()
        nom = td[3].get_text().strip()

        #check if already in DB
        query = "SELECT * from movies where title like '" + title + "'"
        #print(query)
        cursor.execute(query)
        data = cursor.fetchall()

        if len(data) == 0:
            query = f"INSERT into movies(title) values('{title}')"
            cursor.execute(query)
            dbconnection.commit()

            movie_id = cursor.lastrowid

            query = """INSERT into awards(movie_id, awardshow_id, year, awards, nominations)
                        values(%s, %s, %s, %s, %s)"""
            values = (movie_id, 1, year, awards, nom)
            cursor.execute(query, values)
            dbconnection.commit()

cursor.close()
dbconnection.close()
